In [ ]:
import model 
import data 
import matplotlib.pyplot as plt
from importlib import reload
reload(data)
reload(model)

In [ ]:
# for Gunpowder
ZARR_FOLDER = '../../../../data/cells/raw/zarr/old/'
ZARR_FOLDER_TRAIN = '../../../../data/cells/raw/zarr/train/'
ZARR_FOLDER_TEST = '../../../../data/cells/raw/zarr/test/'
gp_batch_size= 12
gp_voxel_shape = [1,1,1]
gp_input_shape= [256, 256,4]
gp_output_shape = [ 256, 256,4]

# For unet
OUTPUT_PATH = '../../../../data/cells/models/'

unet_input_size = (256,256,4)
unet_output_size = 4

In [ ]:
from datetime import date

today = date.today()

d1 = today.strftime("%d_%m_%Y_")
model_name = d1+'unet_v1_cells.hdf5'
model_name

### Check data 

In [ ]:
gen_train_fast = data.generate_fast_training_batch(ZARR_FOLDER, batch_size=gp_batch_size
                                              ,voxel_shape = gp_voxel_shape,
                                              input_shape= gp_input_shape,output_shape = gp_output_shape )
gen_train_fast2 = data.generate_fast_training_batch(ZARR_FOLDER_TRAIN, batch_size=gp_batch_size
                                              ,voxel_shape = gp_voxel_shape,
                                              input_shape= gp_input_shape,output_shape = gp_output_shape )
gen_test_fast = data.generate_fast_training_batch(ZARR_FOLDER_TEST, batch_size=gp_batch_size
                                              ,voxel_shape = gp_voxel_shape,
                                              input_shape= gp_input_shape,output_shape = gp_output_shape )

In [ ]:
im,mask = next(gen_train_fast)

In [ ]:
test_im = im[0]
test_mask = mask[0]

print('Img size: {} {} < {} | Mask size: {} {} <{}'.format(test_im.shape,test_im.dtype,test_im.max()
                                                                   ,test_mask.shape,test_mask.dtype,test_mask.max()))

In [ ]:
plt.imshow(test_im[:,:,2])
plt.show()
plt.imshow(test_mask[:,:,3])
plt.show()

### Model

In [ ]:
unet_model = model.unet(input_size = unet_input_size ,output_size = unet_output_size)

In [ ]:
unet_model.summary()

### Train

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
import os

model_file = os.path.join(OUTPUT_PATH,model_name)
# mean_io_u
model_checkpoint = ModelCheckpoint(model_file, monitor='val_loss',verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001, verbose=1,  cooldown=1)

In [ ]:
history = unet_model.fit_generator(gen_train_fast,validation_data =gen_test_fast,validation_steps=10,steps_per_epoch=50,epochs=10,callbacks=[model_checkpoint,reduce_lr])


### Test

In [ ]:
im,mask = next(gen_train_fast)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(im[0][:,:,2])
# print(im[0])

In [ ]:
results = unet_model.predict(im)

In [ ]:
plt.imshow(results[3][:,:,1])

In [ ]:
today = date.today()

d1 = today.strftime("%d_%m_%Y_")
model_name2 = d1+'unet_v2_cells.hdf5'
model_name2

In [ ]:
unet_model_v2 = model.unet(input_size = unet_input_size ,output_size = unet_output_size)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import os

model_file2 = os.path.join(OUTPUT_PATH,model_name2)
# mean_io_u
model_checkpoint2 = ModelCheckpoint(model_file2, monitor='mean_io_u',verbose=1, save_best_only=True)

In [ ]:
history2 = unet_model_v2.fit_generator(gen_train_fast2, validation_data =gen_test_fast,  steps_per_epoch=50,epochs=200,callbacks=[model_checkpoint2])
